The text source is processed to create 

Used temperature sampling

In [2]:
%reload_ext autoreload
%matplotlib inline

In [3]:
%autoreload 2

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys, io

/home/admin-u4610248/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
path = get_file(
    'nietzsche.txt', 
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
# need to get a source of the content

with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

print('text corpus length: ', len(text))

text corpus length:  600893


In [5]:
chars = sorted(list(set(text)))
numChars = len(chars)
print() # print total number of chars

char_indices = { ch : i for i, ch in enumerate(chars)}
indices_char = { i : ch for i, ch in enumerate(chars)}
# indices_char2 = { i : ch for ch, i in char_indices.items()}
# print(char_indices)
# print(indices_char1)

In [6]:
def create_seq(text, maxLen, step):
    sentences = list()
    next_chars = list()
    for i in range(0, len(text) - maxLen, step):
        sentences.append(text[i: i + maxLen])
        next_chars.append(text[i + maxLen])
    print('nb seq: ', len(sentences))
    return sentences, next_chars

maxLen = 40
step = 3
sentences, next_chars = create_seq(text, maxLen, step)
# TODO: need to use sentences rather than sequen

nb seq:  200285


In [7]:
# Vectorization
x = np.zeros( (len(sentences), maxLen, numChars), dtype=np.bool )
y = np.zeros( (len(sentences), numChars), dtype=np.bool )

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [8]:
# model: a single LSTM
model = Sequential()
model.add(LSTM(units=128, input_shape=(maxLen, numChars)))
model.add(Dense(numChars))
model.add(Activation('softmax'))

### Temperature sampling

The temperature is used to divide the predicted log probabilities before applying the softmax. Lower temperature will cause the model to generate more likely sequences; higher temperature will lead to the model making diverse choices, but the model is prone to make more mistakes.

[karpathy,char-nn](https://github.com/karpathy/char-rnn)  
[stackexchange,temperature-lstm](https://cs.stackexchange.com/questions/79241/what-is-temperature-in-lstm-and-neural-networks-generally)

In [9]:
# sampling an index from a probability array
def sample(preds, temperature=1.0):
    preds = np.asarray(preds)
    preds = np.log(preds) / temperature
    preds = np.exp(preds) / np.sum(np.exp(preds)) # softmax function
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

print(sample([0.01, 0.02132,0.1,0.2,0.3]))

2


In [ ]:
def at_epoch_end(epoch, logs):
    print('\n ---- Generating text after epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxLen - 1)
    
    diversity_range = [0.2, 0.6, 1.0, 1.2]
    for temp in diversity_range:
        print('---- Diversity:', temp)
        
        generated = ''
        sentence = text[start_index: (start_index + maxLen)]
        generated += sentence
        
        print('----- Generating sequence with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(100):
            x_pred = np.zeros((1, maxLen, numChars))
            
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
                
                preds = model.predict(preds)[0] #verbosity default is 0
                next_index = sample(preds, temp)
                next_char = indices_char(next_index)
                
                generated += next_char
                sentence = sentence[1:] + next_char

Need to check out [this](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/)